<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">پیش‌پردازش و پاک‌سازی متن</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
در این بخش، یک تابع سفارشی برای پاک‌سازی متن‌ها تعریف شده است. عملیاتی مانند حذف تگ‌های HTML، لینک‌ها، کاراکترهای تکراری و علائم نگارشی اضافه صورت می‌گیرد.
</p>


In [10]:
import pandas as pd
import numpy as np
import re
import gdown
import gc
from IPython.display import display
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [11]:
gdown.download("https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW", "train_data.csv", quiet =True)


'train_data.csv'

In [12]:
df_train =pd.read_csv("train_data.csv", low_memory =False)

In [13]:
def clean_txt(t):
    if not isinstance(t, str):
        return ""
    t =t.lower()
    t =re.sub(r"<.*?>", " ", t)
    t =re.sub(r"http\S+|www\S+|https\S+", "", t, flags =re.MULTILINE)
    t =re.sub(r"(.)\1{2,}", r"\1\1", t)
    t =re.sub(r"[^a-z0-9\s\!\?\.\']", " ", t)
    t =re.sub(r"\s+", " ", t).strip()
    return t

In [14]:
df_train['summary'] =df_train['summary'].fillna("")
df_train['reviewText'] =df_train['reviewText'].fillna("")
df_train['full_txt'] =df_train['summary'] + " . " + df_train['reviewText']

In [15]:
df_train['clean_txt'] =df_train['full_txt'].apply(clean_txt)
req_cols =['clean_txt', 'overall']
df_train =df_train[req_cols]
gc.collect()
display(df_train.head())
print(df_train.info())

,clean_txt,overall
0,cannot learn . i have an older urc wr7 remote ...,2
1,zero programming needed! miracle!? . first tim...,5
2,works good and programs easy. . got them and o...,4
3,same as twc remote . i got tired of the remote...,5
4,good quality cord . after purchasing cheap cor...,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838944 entries, 0 to 838943
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   clean_txt  838944 non-null  object
 1   overall    838944 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 12.8+ MB
None


<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">نمونه‌گیری و آماده‌سازی لیبل‌ها</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
برای ساخت مدل پایه (Baseline) و تست سریعِ معماری، از یک نمونه‌ی تصادفی ۱۰۰ هزارتایی استفاده کرده‌ایم. همچنین مقادیرِ امتیازها (۱ تا ۵ ستاره) را یکی کم کرده‌ایم تا در بازه‌ی (۰ تا ۴) قرار بگیرند. این کار برای سازگاری کاملِ لیبل‌ها با تابع خطای sparse_categorical_crossentropy در تنسورفلو کاملاً ضروری است.
</p>


In [16]:
df_samp =df_train.sample(n =100000, random_state =42)
y_lbl =df_samp['overall'] -1
x_tr, x_val, y_tr, y_val =train_test_split(df_samp['clean_txt'], y_lbl, test_size =0.2, random_state =42)

<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">استخراج ویژگی با تکنیک TF-IDF</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
به عنوان مدل پایه، به جای استفاده از امبدینگ‌های پیچیده (مانند Word2Vec یا ترنسفورمرها)، از تکنیک کلاسیک و قدرتمند TF-IDF استفاده می‌کنیم. لایه‌ی `TextVectorization` در کراس، ۱۰ هزار کلمه‌ی مهم و پرتکرارِ دیتاسِت را شناسایی کرده و متن خام را مستقیماً به بردارهای عددی تبدیل می‌کند.
</p>


In [17]:
v_lay =tf.keras.layers.TextVectorization(max_tokens =10000, output_mode ='tf_idf')
ds =tf.data.Dataset.from_tensor_slices(x_tr.values).batch(1024)
v_lay.adapt(ds)

<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">معماری شبکه‌ی عصبی پایه</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
معماری مدل شامل یک شبکه‌ی عصبی عمیق (Feed-Forward) ساده با دو لایه‌ی مخفی (۱۲۸ و ۶۴ نورونی) با تابع فعال‌سازی ReLU است. برای جلوگیری از حفظ کردن دیتا توسط مدل، از یک لایه‌ی Dropout با نرخ ۲۰ درصد استفاده شده است. لایه‌ی خروجی نیز با تابع Softmax، احتمال تعلقِ نظر کاربر به هر یک از ۵ کلاس  را محاسبه می‌کند.
</p>


In [18]:
b_mod =tf.keras.Sequential()
b_mod.add(v_lay)
b_mod.add(tf.keras.layers.Dense(128, activation ='relu'))
b_mod.add(tf.keras.layers.Dropout(0.2))
b_mod.add(tf.keras.layers.Dense(64, activation ='relu'))
b_mod.add(tf.keras.layers.Dense(5, activation ='softmax'))

In [19]:
b_mod.compile(optimizer ='adam', loss ='sparse_categorical_crossentropy', metrics =['accuracy'])

In [20]:
b_mod.fit(x_tr.values, y_tr.values, validation_data =(x_val.values, y_val.values), epochs =3, batch_size =1024, verbose =2)

Epoch 1/3
79/79 - 12s - 157ms/step - accuracy: 0.6033 - loss: 1.0426 - val_accuracy: 0.6716 - val_loss: 0.8401
Epoch 2/3
79/79 - 4s - 55ms/step - accuracy: 0.7223 - loss: 0.7182 - val_accuracy: 0.6748 - val_loss: 0.8267
Epoch 3/3
79/79 - 5s - 59ms/step - accuracy: 0.7892 - loss: 0.5545 - val_accuracy: 0.6684 - val_loss: 0.8959


In [21]:
p_raw =b_mod.predict(x_val.values, batch_size =1024)
p_cls =np.argmax(p_raw, axis =1)
f1_m =f1_score(y_val, p_cls, average ='micro')
print("Baseline F1 Score:")
print(f1_m)
b_mod.save_weights('baseline_weights.weights.h5')

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Baseline F1 Score:
0.6684


<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">نتایج مدل پایه</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
مدل پایه‌ی ما توانست به نمره‌ی `F1-Micro` معادل **۶۶.۸٪** دست پیدا کند که برای یک شبکه‌ی ساده‌ی مبتنی بر TF-IDF شروع قابل قبولی است.

**تحلیل لاگ‌های آموزش:**
با بررسی دقیق لاگ‌ها در ۳ ایپاک، متوجه یک پدیده‌ی  رایج می‌شویم: 
خطای آموزش (Train Loss) به شدت در حال کاهش است (از ۱.۰۴ به ۰.۵۵)، اما خطای اعتبارسنجی (Val Loss) پس از ایپاک دوم روند صعودی گرفته (از ۰.۸۲ به ۰.۸۹) و دقت روی دیتای جدید افت کرده است. 
این الگوی رفتاری، نشان‌دهنده‌ی شروعِ پدیده‌ی **بیش‌برازش (Overfitting)** است. این یعنی معماری شبکه‌ی فعلی، ظرفیت درک لحن‌های پیچیده‌ی متون را ندارد و صرفاً در حال حفظ کردنِ دیتای آموزشی است. 
</p>
